# This is an AI ChatBot

In [64]:
#!pip install -U spacy
!python -m spacy download en_core_web_sm

Traceback (most recent call last):
  File "/Users/tthekkum/Documents/LnD/ExternalGit/Projects/ChatBot/venv/lib/python3.8/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/Users/tthekkum/Documents/LnD/ExternalGit/Projects/ChatBot/venv/lib/python3.8/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/Users/tthekkum/Documents/LnD/ExternalGit/Projects/ChatBot/venv/lib/python3.8/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 60] Operation timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tthekkum/Documents/LnD/ExternalGit/Projects/ChatBot/venv/lib/python3.8/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/Users/tthekkum/Documents/LnD/ExternalGit/Projects/ChatBot/venv/lib/python

In [65]:
# import numpy as np
import json
import re
# import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

In [66]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

Import the Intent JSON files

In [67]:
with open('./Intent.json') as f:
    data = json.load(f)

In [68]:
#print(json.dumps(data, indent=4, sort_keys=True))

In [69]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [70]:
# get text and intent title from json data
inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

In [71]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    
    return tokenizer, input_seq

In [72]:
# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)

In [73]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

In [74]:
# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

In [75]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))

input shape: (143, 9) and output shape: (143, 22)


## Build the model

In [76]:
# hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [77]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 512)         66048     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               656384    
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 22)                2838      
Total params: 758,166
Trainable params: 758,166
Non-trainable params: 0
_________________________________________________________________


In [78]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

In [79]:
# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
5/5 [==============================] - 2s 14ms/step - loss: 3.0875 - accuracy: 0.0629
Epoch 2/50
5/5 [==============================] - 0s 16ms/step - loss: 2.6554 - accuracy: 0.2657
Epoch 3/50
5/5 [==============================] - 0s 14ms/step - loss: 1.8728 - accuracy: 0.4965
Epoch 4/50
5/5 [==============================] - 0s 13ms/step - loss: 1.1476 - accuracy: 0.5944

In [80]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

In [ ]:
# chat with bot
print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('Bot: {} -- TYPE: {}'.format(res, typ))
    print()

Note: Enter 'quit' to break the loop.
You: Hi
Bot: They call you <HUMAN>, what can I do for you? -- TYPE: CurrentHumanQuery

You: help
Bot: Bye! Come back again soon. -- TYPE: CourtesyGoodBye

You: what are you doing
Bot: Hi, I am good thank you, how are you? Please tell me your GeniSys user -- TYPE: CourtesyGreeting

